In [1]:
import pandas as pd

C:\Users\HP USER\AppData\Local\Temp\ipykernel_43656\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
import os
file_path = "history.csv"
if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
    df = pd.read_csv(file_path)
else:
    print("Error: The file is empty or does not exist.")


C:\Users\HP USER\AppData\Local\Temp\ipykernel_43656\764340771.py:4: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [54]:
# Get columns A to Z
A_Z_columns = df.iloc[:, :26]
A_Z_columns.head(3)

,id,type_code,is_registered,country_code,currency_code,is_active,class_id,debit_code,last_trade_date,contract_type,...,branch,include_client_consolidation,use_client_address,credit_limit_type,retail_plan,is_spousal,is_arp_locked,arp_pension_origin,language_code,dividend_confirm_code
0,893724,CASH SWEEP,f,NaN,CAD,t,3.0,0,NaN,NaN,...,IAVM,t,f,0.0,NaN,NaN,NaN,NaN,E,0.0
1,1268094,RRIF,t,NaN,CAD,t,5.0,C,2022-07-04,18.0,...,IAVM,t,t,0.0,RI,f,f,NaN,E,0.0
2,606613,SPOUSAL RRSP,t,NaN,CAD,t,5.0,A,2018-07-05,16.0,...,IAVM,f,f,0.0,RS,t,f,NaN,F,0.0


In [33]:
CA_to_end = df.iloc[:, -27:]
CA_to_end.head()

,is_broker_account,is_inventory_account,is_gl_account,is_control_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,...,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,f,f,f,f,t,2021-12-16 00:00:00,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,f,f,f,f,t,2023-01-03 00:00:00,BATCH,f,NaN,0000,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,f,f,f,f,t,2021-03-15 00:00:00,GUERINO,f,NaN,00/0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn
3,f,f,f,f,t,2021-01-04 00:00:00,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,f,f,f,f,t,2023-06-27 00:00:00,H01,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,NaN,NaN,f,Churn


In [14]:
def drop_cols(df, cols):
    df = df.drop(cols, axis=1)
    return df

In [15]:
def boolean_map(df, cols):
    for col in cols:
        df[col] = df[col].fillna('f').map({'t': 1, 'f': 0})
    return df

In [16]:
# Commentary: Standardized value from 0 to 1. Filled blanks with 0.
def parse_time(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
        min_date = df[col].min()
        df[col] = df[col].fillna(min_date)

        df[col] = df[col].astype(int)
        min = df[col].min()
        max = df[col].max()    
        df[col] = (df[col] - min) / (max - min)

    return df

In [55]:
# Filter by columns which we wish to use
A_Z_columns = A_Z_columns[['id', 'type_code', 'class_id', 'inception_date', 'branch', 'include_client_consolidation', 'use_client_address', 'credit_limit_type', 'is_arp_locked', 'dividend_confirm_code']]
A_Z_columns.head(3)

,id,type_code,class_id,inception_date,branch,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code
0,893724,CASH SWEEP,3.0,2017-08-18,IAVM,t,f,0.0,NaN,0.0
1,1268094,RRIF,5.0,2018-11-08,IAVM,t,t,0.0,f,0.0
2,606613,SPOUSAL RRSP,5.0,2009-04-08,IAVM,f,f,0.0,f,0.0


In [56]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Encoding type_code column
A_Z_columns.insert(A_Z_columns.columns.get_loc('type_code') + 1, column='type_code_encoded', value=label_encoder.fit_transform(A_Z_columns['type_code']))
A_Z_columns.head(3)

,id,type_code,type_code_encoded,class_id,inception_date,branch,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code
0,893724,CASH SWEEP,1,3.0,2017-08-18,IAVM,t,f,0.0,NaN,0.0
1,1268094,RRIF,15,5.0,2018-11-08,IAVM,t,t,0.0,f,0.0
2,606613,SPOUSAL RRSP,18,5.0,2009-04-08,IAVM,f,f,0.0,f,0.0


In [57]:
# Parse date column
parse_time(A_Z_columns, ['inception_date'])


TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

In [58]:
# Encoding branch column
A_Z_columns.insert(A_Z_columns.columns.get_loc('branch') + 1, column='branch_encoded', value=label_encoder.fit_transform(A_Z_columns['branch']))
A_Z_columns.head(3)


,id,type_code,type_code_encoded,class_id,inception_date,branch,branch_encoded,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code
0,893724,CASH SWEEP,1,3.0,2017-08-18,IAVM,5,t,f,0.0,NaN,0.0
1,1268094,RRIF,15,5.0,2018-11-08,IAVM,5,t,t,0.0,f,0.0
2,606613,SPOUSAL RRSP,18,5.0,2009-04-08,IAVM,5,f,f,0.0,f,0.0


In [38]:
mapping_columns = ['include_client_consolidation', 'use_client_address', 'is_arp_locked']
A_Z_columns = boolean_map(A_Z_columns, mapping_columns)

In [34]:
# 1. Drop Useless
drop_columns = ['is_broker_account', 'is_gl_account', 'is_control_account', 'is_plan_grandfathered']
CA_to_end = drop_cols(CA_to_end, drop_columns)

In [35]:
# 2. Map Booleans
bool_columns = ['is_inventory_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'is_family_resp', 'is_hrdc_resp']
CA_to_end = boolean_map(CA_to_end, bool_columns)

bool_maybe_columns = ['is_olob', 'visible_in_reports', 'use_original_date_for_payment_calc']
CA_to_end = boolean_map(CA_to_end, bool_maybe_columns)

In [36]:
# 3. Parse Times
time_columns = ['last_maintenance_time', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']

CA_to_end = parse_time(CA_to_end, time_columns)

TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

In [10]:
# 4. Miscellaneous/Unsure
CA_to_end.number_of_beneficiaries.fillna(0, inplace=True)
CA_to_end.label = CA_to_end.label.map({'No Churn': 0, 'Churn': 1})

Looking for guidance on these columns
1. last_maintenance_user: A LOT of different users. Map to numbers?
2. retail_last_maintenance_user: Same as above
3. special_fee_code: numbers, characters, blanks
4. resp_specimen plan: numbers (like ids) and blanks

In [11]:
CA_to_end.head()

,is_inventory_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,plan_effective_date,plan_end_date,is_resp,...,is_family_resp,is_hrdc_resp,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,0,1,0.811210,T80,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.835465,0,0.000000,NaN,0,1
1,0,1,0.929165,BATCH,0,NaN,0000,0.826910,0.000000,0,...,0,0,NaN,0.0,0.860734,0,0.954733,BATCH,1,1
2,0,1,0.726209,GUERINO,0,NaN,00/0,0.480036,0.691887,0,...,0,0,NaN,0.0,0.860734,0,0.633734,GUERINO,0,1
3,0,1,0.704650,T80,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.835465,0,0.000000,NaN,1,1
4,0,1,0.983061,H01,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.836450,0,0.000000,NaN,0,1
